<a href="https://colab.research.google.com/github/pravsels/DistilLM/blob/main/manim/HF_Training_Script_mp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install datasets transformers accelerate peft bitsandbytes wandb flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import transformers
from datasets import load_dataset
import math
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [5]:
hf_dataset = load_dataset('pravsels/manim_pile')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7751 [00:00<?, ? examples/s]

In [6]:
hf_dataset['train'], hf_dataset['eval'] = hf_dataset['train'].train_test_split(.1).values()

hf_dataset

DatasetDict({
    train: Dataset({
        features: ['metadata', 'text'],
        num_rows: 6975
    })
    eval: Dataset({
        features: ['metadata', 'text'],
        num_rows: 776
    })
})

In [7]:
hf_dataset['train'][11]

{'metadata': {'dataset_name': 'pravsels/manim_ManimCommunity_issues',
  'filename': 'N/A'},
 'text': "Don't use deprecated methods in deprecation.py : ## Changelog / Overview\r\n<!-- Optional (Recommended): a detailed overview of the PR for the upcoming\r\nrelease's changelog entry. Useful for when the PR title isn't enough. \r\n\r\nDO NOT REMOVE THE FOLLOWING CHANGELOG LINES, EVEN IF YOU DON'T USE THEM.-->\r\n<!--changelog-start-->\r\n\r\n<!--changelog-end-->\r\n\r\n- And fix a missing r-string there\r\n- Use `inspect.signature` rather than `inspect.getargspec`.\r\n\r\n## Motivation\r\nThere were warning there when running pytest, fixing that.\r\n\r\n## Explanation for Changes\r\nFix some warning.\r\n\r\n## Checklist\r\n- [x] I have read the [Contributing Guidelines](https://docs.manim.community/en/latest/contributing.html)\r\n- [x] I have written a descriptive PR title (see top of PR template for examples)\r\n- [x] I have written a changelog entry for the PR or deem it unnecessary\r\

In [8]:
hf_dataset['eval'][11]

{'metadata': {'dataset_name': 'Edoh/manim_python', 'filename': 'N/A'},
 'text': 'Create a regular pentagon with side length 2 and color it purple. : from manim import * class MyScene(Scene): def construct(self): pentagon = RegularPolygon(n=5, radius=2, color=PURPLE) self.add(pentagon)'}

In [9]:
base_model = 'deepseek-ai/deepseek-coder-6.7b-instruct'
# max_context_size= 16384
# setting context length of out inputs
# block_size = tokenizer.model_max_length

block_size = 1024     # context len of input
# using the same tokenizer that was used for training the model
tokenizer = AutoTokenizer.from_pretrained(base_model,
                                          padding_side='left',
                                          use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
import functools
import numpy as np

# this is expensive so we cache it
@functools.lru_cache(maxsize=None)
def get_fim_token_ids(tokenizer):
  if "deepseek-coder" in tokenizer.name_or_path:
    return (
        tokenizer.bos_token_id,
        tokenizer.encode("<｜fim▁hole｜>", add_special_tokens=False)[0],
        tokenizer.encode("<｜fim▁begin｜>", add_special_tokens=False)[0],
        tokenizer.encode("<｜fim▁end｜>", add_special_tokens=False)[0],
        tokenizer.encode("<pad>", add_special_tokens=False)[0],
    )

def _bos_token_processing(prefix_token_list, bos_token):
  if bos_token is not None:
    # add the BOS token to the beginning of the list
    prefix_token_list.insert(0, bos_token)

  return prefix_token_list

def permute(
    sample,
    np_rng,
    suffix_tok_id,
    prefix_tok_id,
    middle_tok_id,
    pad_tok_id,
    fim_rate=0.5,
    fim_spm_rate=0.5,
    truncate_or_pad=False,
    bos_token_id=None,
):
    """
    Take in a sample (list of tokens) and perform a FIM transformation on it with a probability of fim_rate, using two FIM modes:
    PSM and SPM (with a probability of fim_spm_rate).
    """

    if np_rng.binomial(1, fim_rate):
        boundaries = list(np_rng.randint(low=0, high=len(sample) + 1, size=2))
        boundaries.sort()

        prefix = np.array(sample[: boundaries[0]], dtype=np.int64)
        middle = np.array(sample[boundaries[0] : boundaries[1]], dtype=np.int64)
        suffix = np.array(sample[boundaries[1] :], dtype=np.int64)

        if truncate_or_pad:
            new_length = suffix.shape[0] + prefix.shape[0] + middle.shape[0] + 3
            diff = new_length - len(sample)
            if diff > 0:
                if suffix.shape[0] <= diff:
                    return sample, np_rng
                suffix = suffix[: suffix.shape[0] - diff]
            elif diff < 0:
                suffix = np.concatenate([suffix, np.full((-1 * diff), pad_tok_id)])

        if np_rng.binomial(1, fim_spm_rate):
            prefix_special_tokens = _bos_token_processing(
                [prefix_tok_id, suffix_tok_id], bos_token_id
            )
            # SPM (variant 2 from FIM paper)
            new_sample = np.concatenate(
                [
                    prefix_special_tokens,
                    suffix,
                    [middle_tok_id],
                    prefix,
                    middle,
                ]
            )
        else:
            prefix_special_tokens = _bos_token_processing([prefix_tok_id], bos_token_id)
            # PSM
            new_sample = np.concatenate(
                [
                    prefix_special_tokens,
                    prefix,
                    [suffix_tok_id],
                    suffix,
                    [middle_tok_id],
                    middle,
                ]
            )
    else:
        # don't do FIM preproc
        new_sample = sample
    return list(new_sample), np_rng

In [11]:
def tokenize_func(examples):
  return tokenizer(examples['text'],
                   max_length=block_size,
                   truncation=True)

In [12]:
tokenized_dataset = hf_dataset.map(tokenize_func,
                                   batched=True,
                                   num_proc=8,
                                   remove_columns=['text', 'metadata'])

Map (num_proc=8):   0%|          | 0/6975 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/776 [00:00<?, ? examples/s]

In [13]:
(
  bos_token_id,
  suffix_tok_id,
  prefix_tok_id,
  middle_tok_id,
  pad_tok_id,
) = get_fim_token_ids(tokenizer)

In [14]:
def apply_fim_permutation(examples):
    all_token_ids = []
    for tokenized_input in examples['input_ids']:
        permuted_input, np_rpg = permute(
                                          tokenized_input,
                                          np.random.RandomState(seed=0),
                                          suffix_tok_id,
                                          prefix_tok_id,
                                          middle_tok_id,
                                          pad_tok_id,
                                          fim_rate=0.5,
                                          fim_spm_rate=0.5,
                                          truncate_or_pad=False,
                                          bos_token_id=bos_token_id,
                                        )
        all_token_ids.append(permuted_input + [tokenizer.eos_token_id])
    return {'input_ids': all_token_ids}

tokenized_dataset = tokenized_dataset.map(apply_fim_permutation,
                                          num_proc=8,
                                          batched=True)

Map (num_proc=8):   0%|          | 0/6975 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/776 [00:00<?, ? examples/s]

In [16]:
print(tokenized_dataset)
print(tokenized_dataset['train'][0])
print(tokenized_dataset['eval'][0])

In [17]:
def group_text_func(examples):
  # for every key ('train', 'test' etc.), concatenate the texts

  concat_examples = {key: sum(examples[key], []) for key in examples.keys()}
  total_len = len(concat_examples[list(examples.keys())[0]])
  total_len = (total_len // block_size) * block_size

  # split by chunks of block_size
  result = {
      key: [text[i : i + block_size] for i in range(0, total_len, block_size)] for key, text in concat_examples.items()
  }
  result['labels'] = result['input_ids'].copy()

  return result

In [18]:
lm_dataset = tokenized_dataset.map(group_text_func,
                                   batched=True,
                                   batch_size=128,
                                   num_proc=8)
lm_dataset

Map (num_proc=8):   0%|          | 0/6975 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/776 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3701
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 418
    })
})

In [19]:
print('input ids : ', lm_dataset['train'][0]['input_ids'][:5])
print('attention mask : ', lm_dataset['train'][0]['attention_mask'][:5])
print('label ids : ', lm_dataset['train'][0]['labels'][:5])

In [20]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj"],
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [21]:
model = AutoModelForCausalLM.from_pretrained(
                                              base_model,
                                              device_map="auto",
                                              torch_dtype=torch.bfloat16,
                                              attn_implementation="flash_attention_2",
                                              # load_in_8bit=True
                                             )

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [24]:
model.add_adapter(peft_config,
                  adapter_name="manim_adapter")

In [25]:
import numpy as np
def count_trainable_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    total_params = sum([np.prod(p.size()) for p in model.parameters()])
    return params/total_params

count_trainable_parameters(model) * 100

0.49533656825843125

In [26]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [27]:
training_args = TrainingArguments(
  f"{base_model}_finetuned_manim_pile_context_FIM_{block_size}",
  evaluation_strategy = "epoch",
  num_train_epochs=5,
  learning_rate=2e-5,
  per_device_train_batch_size=1,
  per_device_eval_batch_size=1,
  gradient_accumulation_steps=8,
  weight_decay=0.01,
  report_to="wandb",
  run_name=f"{base_model.split('/')[-1]}_manim_pile_context_FIM_{block_size}",
  push_to_hub=True
)

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset['train'],
    eval_dataset=lm_dataset['eval']
)

In [ ]:
trainer.train()

wandb: Currently logged in as: pravsels. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity score : {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
model.save_pretrained('./')

In [ ]:
trainer.push_to_hub()

In [ ]:
base_model = 'deepseek-ai/deepseek-coder-6.7b-instruct'
model_name = 'pravsels/deepseek-coder-6.7b-instruct_finetuned_manim_pile_context_1024'
tokenizer = AutoTokenizer.from_pretrained(base_model,
                                          padding_side='left',
                                          use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_model,
                                             device_map="auto",
                                             torch_dtype=torch.bfloat16,
                                             attn_implementation="flash_attention_2",
                                             load_in_4bit=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.load_adapter(model_name)

In [ ]:
gen_context_len = 512

sys_prompt = "you a helpful AI agent that generates math animation code with the library Manim. you shouldn't be too verbose and also your your generation once the program ends. "

input_text = "write code to transition a blue circle to a red square"
inputs = tokenizer(sys_prompt + input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=gen_context_len)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


RuntimeError: FlashAttention only supports Ampere GPUs or newer.